# Tiled Image - Infer on whole 3D but stack 2D slices.

In [1]:
def calc_dice(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    numerator = 2.0 * np.sum(np.round(y_true)*np.round(y_pred))
    denominator = np.sum(np.round(y_true)) + np.sum(np.round(y_pred))
    if denominator == 0:
       return 1.0
    else:
       return numerator / denominator

def calc_slicewise_average_dice(y_true_all, y_pred_all):
    
    num_slices = y_true_all.shape[0]
    dice_slice = np.zeros((num_slices))
    
    for idx in range(num_slices):
        
        y_true = y_true_all[idx].flatten()
        y_pred = y_pred_all[idx].flatten()

        numerator = 2.0 * np.sum(np.round(y_true)*np.round(y_pred))
        denominator = np.sum(np.round(y_true)) + np.sum(np.round(y_pred))
        if denominator == 0:
           dice_slice[idx] = 1.0
        else:
           dice_slice[idx] = numerator / denominator
        
    return np.mean(dice_slice)

def calc_soft_dice(y_true, y_pred, smooth=1):
    """
    Sorensen Dice coefficient
    """
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    numerator = 2.0 * np.sum(y_true * y_pred) + smooth
    denominator = np.sum(y_true) + np.sum(y_pred) + smooth
    coef = numerator / denominator

    return coef

def dice_coef(y_true, y_pred, axis=(1, 2), smooth=1):
    """
    Sorenson (Soft) Dice
    \frac{  2 \times \left | T \right | \cap \left | P \right |}{ \left | T \right | +  \left | P \right |  }
    where T is ground truth mask and P is the prediction mask
    """
    intersection = tf.reduce_sum(y_true * y_pred, axis=axis)
    union = tf.reduce_sum(y_true + y_pred, axis=axis)
    numerator = tf.constant(2.) * intersection + smooth
    denominator = union + smooth
    coef = numerator / denominator

    return tf.reduce_mean(coef)


def dice_coef_loss(target, prediction, axis=(1, 2), smooth=1.):
    """
    Sorenson (Soft) Dice loss
    Using -log(Dice) as the loss since it is better behaved.
    Also, the log allows avoidance of the division which
    can help prevent underflow when the numbers are very small.
    """
    intersection = tf.reduce_sum(prediction * target, axis=axis)
    p = tf.reduce_sum(prediction, axis=axis)
    t = tf.reduce_sum(target, axis=axis)
    numerator = tf.reduce_mean(intersection + smooth)
    denominator = tf.reduce_mean(t + p + smooth)
    dice_loss = -tf.log(2.*numerator) + tf.log(denominator)

    return dice_loss


def combined_dice_ce_loss(y_true, y_pred, axis=(1, 2), smooth=1.,
                          weight=0.9):
    """
    Combined Dice and Binary Cross Entropy Loss
    """
    return weight*dice_coef_loss(y_true, y_pred, axis, smooth) + \
        (1-weight)*K.losses.binary_crossentropy(y_true, y_pred)


def plot_results(imgs_validation, msks_validation, pred_validation, pred_validation_middle, slice_no, filename):
    """
    Plot the predictions for slice_no
    """
    
    import matplotlib.gridspec as gridspec
    
    import ntpath
    filename = ntpath.basename(filename)


    gs1 = gridspec.GridSpec(2, 4)
    gs1.update(wspace=0.2, hspace=0.005) # set the spacing between axes. 


    img = imgs_validation[[slice_no], ]
    msk = msks_validation[[slice_no], ]
    
    pred_mask = pred_validation[[slice_no], ]
    pred_mask_middle = pred_validation_middle[[slice_no], ]
    
    dice_score = calc_dice(pred_mask, msk)
    
    dice_score_middle = calc_dice(pred_mask_middle, msk)

    
    plt.figure(figsize=(20,15))
    plt.subplot(gs1[0])
    plt.imshow(img[0,:,:,0], cmap="bone", origin="lower")
    plt.title("MRI\n{}".format(filename), fontsize=18)
    
    plt.subplot(gs1[1])
    plt.imshow(msk[0,:,:,0], origin="lower", cmap="gray")
    plt.title("Ground truth", fontsize=18)
    
    plt.subplot(gs1[2])
    plt.imshow(pred_mask[0,:,:,0], origin="lower", cmap="gray")
    plt.title("Prediction\nDice = {:.4f}".format(dice_score), fontsize=18)
    
    plt.subplot(gs1[3])
    plt.imshow(msk[0,:,:,0]-pred_mask[0,:,:,0], cmap="gray", origin="lower")
    plt.title("Difference", fontsize=18)
    
    plt.subplot(gs1[4])
    plt.imshow(img[0,:,:,0], cmap="bone", origin="lower")
    plt.title("MRI\n{}".format(filename), fontsize=18)
    
    plt.subplot(gs1[5])
    plt.imshow(msk[0,:,:,0], origin="lower", cmap="gray", )
    plt.title("Ground truth", fontsize=18)
   
    plt.subplot(gs1[6])
    plt.imshow(pred_mask_middle[0,:,:,0], cmap="gray", origin="lower")
    plt.title("Prediction Middle\nDice = {:.4f}".format(dice_score_middle), fontsize=18)
    
    plt.subplot(gs1[7])
    plt.imshow(pred_mask_middle[0,:,:,0]-msk[0,:,:,0], cmap="gray", origin="lower")
    plt.title("Difference", fontsize=18)
    

In [2]:
!pwd

/home/bduser/ravi/unet-ravi/2D


In [3]:
def plot_tiles(pred_validation_middle, pred_validation_tl, pred_validation_tr, 
               pred_validation_bl, pred_validation_br, slice_no, filename):

    import matplotlib.gridspec as gridspec
    
    import ntpath
    filename = ntpath.basename(filename)


    gs1 = gridspec.GridSpec(3, 3)
    gs1.update(wspace=0.001, hspace=-0.02) # set the spacing between axes. 
    
    
    plt.figure(figsize=(20,15))
    
    # Remember that we flip the image to show it in the correct orientation
    
    plt.subplot(gs1[0])
    plt.imshow(pred_validation_bl[slice_no,:,:,0], origin="lower", cmap="gray")
    
    plt.subplot(gs1[2])
    plt.imshow(pred_validation_br[slice_no,:,:,0], origin="lower", cmap="gray")
    
    plt.subplot(gs1[4])
    plt.imshow(pred_validation_middle[slice_no,:,:,0], origin="lower", cmap="gray")
    plt.title("{}".format(filename), fontsize=18)
    
    plt.subplot(gs1[6])
    plt.imshow(pred_validation_tl[slice_no,:,:,0], origin="lower", cmap="gray")
    
    plt.subplot(gs1[8])
    plt.imshow(pred_validation_tr[slice_no,:,:,0], origin="lower", cmap="gray")
    
    

In [4]:
import os

import numpy as np
import tensorflow as tf
import keras as K
import settings
import argparse
import h5py
import nibabel as nib

import matplotlib.pyplot as plt

args = argparse.Namespace()
args.data_path="/home/bduser/tony/data/decathlon/Task01_BrainTumour/"
args.output_path="/home/bduser/tony/unet_tiling/"
args.inference_filename="unet_decathlon_4_8814_128x128_randomcrop.hdf5"
args.intraop_threads=5
args.interop_threads=2

CONFIG = tf.ConfigProto(
    inter_op_parallelism_threads=args.interop_threads,
    intra_op_parallelism_threads=args.intraop_threads)

SESS = tf.Session(config=CONFIG)
K.backend.set_session(SESS)

model_fn = os.path.join(args.output_path, args.inference_filename)


# Load model
model = K.models.load_model(model_fn, custom_objects={
    "combined_dice_ce_loss": combined_dice_ce_loss,
    "dice_coef_loss": dice_coef_loss,
    "dice_coef": dice_coef})



Using TensorFlow backend.


In [5]:
def create_five_patch_divisor(x,y):
    
    cropx = 128
    cropy = 128

    data = np.zeros((240,240))
    data[0:cropx, 0:cropy] = 1
    tl = data

    data = np.zeros((240,240))
    data[0:cropx, y-cropy:y] = 1
    tr = data

    data = np.zeros((240,240))
    data[x-cropx:x, 0:cropy] = 1
    bl = data

    data = np.zeros((240,240))
    data[x-cropx:x, y-cropy:y] = 1
    br = data

    data = np.zeros((240,240))
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    data[starty:starty+cropy,startx:startx+cropx] = 1
    cc = data

   
    return tl + tr + bl + br + cc
    

def crop_tl(img, cropx, cropy):
    _, x, y, _ = img.shape
    return img[:, 0:cropx, 0:cropy]

def crop_tr(img, cropx, cropy):
    _, x, y, _ = img.shape
    return img[:, 0:cropx, y-cropy:y]

def crop_bl(img, cropx, cropy):
    _, x, y, _ = img.shape
    return img[:, x-cropx:x, 0:cropy]

def crop_br(img, cropx, cropy):
    _, x, y, _ = img.shape
    return img[:, x-cropx:x, y-cropy:y]

def crop_c(img, cropx, cropy):
    _, y, x, _ = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[:,starty:starty+cropy,startx:startx+cropx]


In [6]:
import json
json_filename = os.path.join(args.data_path, "dataset.json")

try:
    with open(json_filename, "r") as fp:
        experiment_data = json.load(fp)
except IOError as e:
    print("File {} doesn't exist. It should be part of the "
          "Decathlon directory".format(json_filename))
        
numFiles = experiment_data["numTraining"]
idxList = np.arange(numFiles)  # List of file indices


np.random.seed(816)
randomIdx = np.random.random(numFiles)  # List of random numbers
# Random number go from 0 to 1. So anything above
# self.train_split is in the validation list.
trainIdx = idxList[randomIdx < 0.85]

listIdx = idxList[randomIdx >= 0.85]
randomIdx = np.random.random(len(listIdx))  # List of random numbers
validateIdx = listIdx[randomIdx >= 0.5]
testIdx = listIdx[randomIdx < 0.5]
        
imgFiles = {}
mskFiles = {}

for idx, fileIdx in enumerate(testIdx):
    imgFiles[idx] = os.path.join(args.data_path,
                                 experiment_data["training"][fileIdx]["image"])
    mskFiles[idx] = os.path.join(args.data_path,
                                 experiment_data["training"][fileIdx]["label"])


In [7]:
def get_data(idx):
    
    filename = imgFiles[idx]
    imgs_validation = np.array(nib.load(imgFiles[idx]).dataobj)
    imgs_validation = np.transpose(imgs_validation, [2, 1, 0, 3])

    msk = np.array(nib.load(mskFiles[idx]).dataobj)
    msk[msk > 0] = 1.0
    msk = np.expand_dims(msk, -1)

    msks_validation = np.transpose(msk, [2, 1, 0, 3])

    _, x, y, _ = imgs_validation.shape
    five_patch_divisor = create_five_patch_divisor(x, y)
    
    return imgs_validation, msks_validation, five_patch_divisor, filename


In [8]:
def predict(imgs_validation, msks_validation, five_patch_divisor):

    from tqdm import tqdm

    cropx = 128
    cropy = 128

    elwise_round = np.vectorize(round, otypes=[np.int])

    pred_validation = np.zeros_like(msks_validation)
    pred_validation_middle = np.zeros_like(msks_validation)
    pred_validation_tl = np.zeros_like(msks_validation)
    pred_validation_tr = np.zeros_like(msks_validation)
    pred_validation_bl = np.zeros_like(msks_validation)
    pred_validation_br = np.zeros_like(msks_validation)

    for img_id in range(imgs_validation.shape[0]):

        full_img = imgs_validation[[img_id], ]
        full_msk = msks_validation[[img_id], ]
        _, x, y, _ = full_img.shape

        tl_img = crop_tl(full_img, cropx, cropy)
        tl_msk = crop_tl(full_msk, cropx, cropy)

        pred_tl_msk = model.predict(tl_img)
        pred_tl_mask_placehld = np.zeros((240,240))
        pred_tl_mask_placehld[0:cropx, 0:cropy]=pred_tl_msk[0][:,:,0]

        tr_img = crop_tr(full_img, cropx, cropy)
        tr_msk = crop_tr(full_msk, cropx, cropy)

        pred_tr_msk = model.predict(tr_img)
        pred_tr_mask_placehld = np.zeros((240,240))
        pred_tr_mask_placehld[0:cropx, y-cropy:y]=pred_tr_msk[0][:,:,0]

        bl_img = crop_bl(full_img, cropx, cropy)
        bl_msk = crop_bl(full_msk, cropx, cropy)

        pred_bl_msk = model.predict(bl_img)
        pred_bl_mask_placehld = np.zeros((240,240))
        pred_bl_mask_placehld[x-cropx:x, 0:cropy]=pred_bl_msk[0][:,:,0]

        br_img = crop_br(full_img, cropx, cropy)
        br_msk = crop_br(full_msk, cropx, cropy)

        pred_br_msk = model.predict(br_img)
        pred_br_mask_placehld = np.zeros((240,240))
        pred_br_mask_placehld[x-cropx:x, y-cropy:y]=pred_br_msk[0][:,:,0]

        cc_img = crop_c(full_img, cropx, cropy)
        cc_msk = crop_c(full_msk, cropx, cropy)

        pred_cc_msk = model.predict(cc_img)
        pred_cc_mask_placehld = np.zeros((240,240))
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)  
        pred_cc_mask_placehld[startx:startx+cropx, starty:starty+cropy]=pred_cc_msk[0][:,:,0]

    #     # Round after averaging
    #     pred_full_msk = pred_tl_mask_placehld + pred_tr_mask_placehld + pred_bl_mask_placehld + pred_br_mask_placehld + pred_cc_mask_placehld
    #     pred_full_msk_avg = pred_full_msk // five_patch_divisor
    #     pred_full_msk_avg = elwise_round(pred_full_msk_avg)


        # Round before averaging
        pred_full_msk = []
        pred_full_msk = elwise_round(pred_tl_mask_placehld) + elwise_round(pred_tr_mask_placehld) + elwise_round(pred_bl_mask_placehld) + elwise_round(pred_br_mask_placehld) + elwise_round(pred_cc_mask_placehld)
        pred_full_msk_avg = pred_full_msk // five_patch_divisor

        pred_validation[img_id] = np.expand_dims(pred_full_msk_avg, -1)

        pred_validation_middle[img_id] = np.expand_dims(elwise_round(pred_cc_mask_placehld), -1)

        pred_validation_tl[img_id] = np.expand_dims(elwise_round(pred_tl_mask_placehld), -1)
        pred_validation_tr[img_id] = np.expand_dims(elwise_round(pred_tr_mask_placehld), -1)
        pred_validation_bl[img_id] = np.expand_dims(elwise_round(pred_bl_mask_placehld), -1)
        pred_validation_br[img_id] = np.expand_dims(elwise_round(pred_br_mask_placehld), -1)
        
    return pred_validation, pred_validation_middle, pred_validation_tl, pred_validation_tr, pred_validation_bl, pred_validation_br

    

In [9]:
%%time
from tqdm import tqdm
dice_results = []
#for img_file_id in tqdm(range(2)):
for img_file_id in tqdm(imgFiles):
    imgs_validation, msks_validation, five_patch_divisor, filename = get_data(img_file_id)
    pred_validation, pred_validation_middle, pred_validation_tl, pred_validation_tr, pred_validation_bl, pred_validation_br = predict(imgs_validation, msks_validation, five_patch_divisor)
    
    dice_3d = calc_dice(msks_validation, pred_validation)
    dice_slicewise = calc_slicewise_average_dice(msks_validation, pred_validation)
    
    dice_results.append([img_file_id, dice_3d, dice_slicewise, filename])

100%|██████████| 46/46 [21:20<00:00, 27.82s/it]

CPU times: user 4h 32min 12s, sys: 3h 8min 58s, total: 7h 41min 10s
Wall time: 21min 20s


In [10]:
print(dice_results)

[[0, 0.7648350033792718, 0.67739689123437, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_206.nii.gz'], [1, 0.2608519949272723, 0.29727809965751495, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_017.nii.gz'], [2, 0.7739124227081525, 0.7122249217778268, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_074.nii.gz'], [3, 0.2548528365818769, 0.33514050407752216, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_023.nii.gz'], [4, 0.3894291467422988, 0.43259384521987326, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_007.nii.gz'], [5, 0.0804246966452534, 0.17630015984748534, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_176.nii.gz'], [6, 0.8039096842257687, 0.5578833306467972, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./imagesTr/BRATS_015.nii.gz'], [7, 0.25889282414536496, 0.517219958920164, '/home/bduser/tony/data/decathlon/Task01_BrainTumour/./im

In [11]:
np.savetxt("dice_res_tiledimage_imgid_3d-dice_slicewise-dice.csv", dice_results, delimiter=",", fmt="%s")


In [12]:
dice_results = np.asarray(dice_results)
dice_3d = dice_results[:,1]
dice_slicewise = dice_results[:,2]
print("Tiled Image Inference Results")
print("Avg of 45 patients' 3D Dice = {:.4f}".format(np.average(dice_3d.astype(float))))
print("Avg of 45 patients' Slicewise Average Dice = {:.4f}".format(np.average(dice_slicewise.astype(float))))

Tiled Image Inference Results
Avg of 45 patients' 3D Dice = 0.5299
Avg of 45 patients' Slicewise Average Dice = 0.5619


In [ ]:
pred_validation, pred_validation_middle, pred_validation_tl, pred_validation_tr, pred_validation_bl, pred_validation_br = predict(imgs_validation, msks_validation, five_patch_divisor)

In [ ]:
print("3D Dice = {:.4f}".format(calc_dice(msks_validation, pred_validation)))
print("Slicewise Average Dice = {:.4f}".format(calc_slicewise_average_dice(msks_validation, pred_validation)))

In [ ]:
plot_results(imgs_validation, msks_validation, pred_validation, pred_validation_middle, 96, filename)

In [ ]:
plot_tiles(pred_validation_middle, pred_validation_tl, pred_validation_tr, pred_validation_bl, pred_validation_br, 96, filename)